In [3]:
# !pip install bert-extractive-summarizer
# !pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow-gpu==2.2

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

from pathlib import Path

--------------------------------------------------

DATA LOADER

In [4]:
import pandas as pd
check_path = "/content/train.csv"
my_file = Path(check_path)
flag = 0
if flag and not my_file.is_file():
    check_path = "/content/AMICorpusXML/data/ami-summary/abstractive/ES2004a.abssumm.txt"

    my_file = Path(check_path)
    if not my_file.is_file():
        if not Path("/content/AMICorpusXML/data").is_dir():
            ! git clone https://github.com/gcunhase/AMICorpusXML
        ! python /content/AMICorpusXML/main_obtain_meeting2summary_data.py --summary_type abstractive
    else:
        print("Data already prepared... Importing\n")

    val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
    train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
                ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
                TS3011, TS3012""".split(",")
    test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(",")

    story_directory = r'/content/AMICorpusXML/data/ami-transcripts-stories/abstractive'
    sum_directory = r'/content/AMICorpusXML/data/ami-summary/abstractive'

    story_val = []
    for filename in os.listdir(story_directory):
        if filename.endswith(".story"):
            for each in val:
                if each.strip() in str(filename):
                    story_val.append(filename)

    def data_read(f1):
        with open(f1, 'r') as file:
            data1 = file.read().replace('\n', '')
        return data1

    val_data = []
    val_summ = []
    for each in story_val:
        a = each.split('.')[0]
        story_file = story_directory+'/'+each
        summ_file = sum_directory+'/'+a+".abssumm.txt"
        data = data_read(story_file)
        val_data.append(data)
        data = data_read(summ_file)
        val_summ.append(data)
else:
    train = pd.read_csv("./content/train.csv", sep='\t')
    train = train.drop(['Unnamed: 0'],axis=1)
    test = pd.read_csv("./content/test.csv", sep='\t')
    test = test.drop(['Unnamed: 0'],axis=1)
    val = pd.read_csv("./content/val.csv", sep='\t')
    val = val.drop(['Unnamed: 0'],axis=1)

In [5]:
train.head()

,inputs,targets
0,'Kay. I'll go first. No Nathan's fine. It's ei...,The Industrial Designer presented an analysis ...
1,Um I'm Craig and I'm User Interface. Yeah. Wel...,The project manager introduced the upcoming pr...
2,How was lunch? Mm. Why? Mm-hmm. W we can use s...,The project manager stated the agenda and set ...
3,"Uh we don't have any changes, do we? So no. It...",The UI and ID presented a prototype drawing of...
4,Hold that. Okay. Okay. Mm. Mm-hmm. Mm-hmm. Mm....,The meeting begins with the group trying to re...


----------------------

## EXTRACTIVE SUMMARY GENERATOR

BERTSUM

In [6]:
from summarizer import Summarizer
from tqdm import tqdm
model = Summarizer()

In [7]:
# test = test[]

In [8]:
ext_summary = []
# model.eval()
for index, row in tqdm(test.iterrows(), ):
    input = row['inputs']
    target = row['targets']
#     print(input)
    result = model(input, max_length=70, num_sentences=2)
    summ = "".join(result)
    ext_summary.append(summ)

test['ext_summary'] = ext_summary

20it [00:49,  2.47s/it]


In [9]:
test.to_csv('test_ext.csv', sep='\t')
test.head()

,inputs,targets,ext_summary
0,"Thanks. Yeah, that's probably a better one, to...",The Project Manager reviewed the minutes from ...,"Yeah, that's probably a better one, to discuss..."
1,Good afternoon. So Hello. No problem. Okay. Th...,In the detailed design meeting the team create...,In this meeting we will discuss our final desi...
2,Okay. Everybody found his place again? Yeah? T...,The project manager stated the agenda and the ...,Uh now we're going um into the functional desi...
3,Hi. Oh. Good morning. I'm Juergen Toffs. I'm t...,The participants introduced themselves and the...,Or You have to draw uh push hard on the pen or...
4,"Morning. Yep. My name's Frank. Thank you. Hmm,...",The project manager introduced himself to the ...,"Can't say I paid much attention to it, but Oh ..."


## EXTRACTIVE TO ABSTRACTIVE

GPT2 Decoder

In [10]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
modelgpt2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [13]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')
    summ = modelgpt2.generate(input_ids, max_length=200)
    summ_decoded = tokenizer.decode(summ[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['abs_summary'] = abs_summary

20it [03:57, 11.86s/it]


BEAM

In [16]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')
    beam_output = modelgpt2.generate(
        input_ids, 
        max_length=200, 
        num_beams=5, 
        early_stopping=True
    )

    summ_decoded =tokenizer.decode(beam_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['beam_abs_summary'] = abs_summary

20it [03:58, 11.93s/it]


In [17]:
test.head()

,inputs,targets,ext_summary,abs_summary,beam_abs_summary
0,"Thanks. Yeah, that's probably a better one, to...",The Project Manager reviewed the minutes from ...,"Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss..."
1,Good afternoon. So Hello. No problem. Okay. Th...,In the detailed design meeting the team create...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...
2,Okay. Everybody found his place again? Yeah? T...,The project manager stated the agenda and the ...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...
3,Hi. Oh. Good morning. I'm Juergen Toffs. I'm t...,The participants introduced themselves and the...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...
4,"Morning. Yep. My name's Frank. Thank you. Hmm,...",The project manager introduced himself to the ...,"Can't say I paid much attention to it, but Oh ...","Can't say I paid much attention to it, but Oh ...","Can't say I paid much attention to it, but Oh ..."


TOP P

In [18]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')

    # set seed to reproduce results. Feel free to change the seed though to get different results
    tf.random.set_seed(0)
    # deactivate top_k sampling and sample only from 92% most likely words
    sample_output = modelgpt2.generate(
        input_ids, 
        do_sample=True, 
        max_length=200, 
        top_p=0.92, 
        top_k=0
    )
    summ_decoded = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)
test['top_P_abs_summary'] = abs_summary

20it [04:29, 13.46s/it]


In [19]:
test.to_csv('test_ext.csv', sep='\t')
test.head()

,inputs,targets,ext_summary,abs_summary,beam_abs_summary,top_P_abs_summary
0,"Thanks. Yeah, that's probably a better one, to...",The Project Manager reviewed the minutes from ...,"Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss...","Yeah, that's probably a better one, to discuss..."
1,Good afternoon. So Hello. No problem. Okay. Th...,In the detailed design meeting the team create...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...,In this meeting we will discuss our final desi...
2,Okay. Everybody found his place again? Yeah? T...,The project manager stated the agenda and the ...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...,Uh now we're going um into the functional desi...
3,Hi. Oh. Good morning. I'm Juergen Toffs. I'm t...,The participants introduced themselves and the...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...,Or You have to draw uh push hard on the pen or...
4,"Morning. Yep. My name's Frank. Thank you. Hmm,...",The project manager introduced himself to the ...,"Can't say I paid much attention to it, but Oh ...","Can't say I paid much attention to it, but Oh ...","Can't say I paid much attention to it, but Oh ...","Can't say I paid much attention to it, but Oh ..."


ROUGE

In [20]:
from rouge_score import rouge_scorer

In [21]:
def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    # print(scores)
    return scores

In [22]:
import numpy as np

rouge1 = []
rouge2 = []
# word_capt = []
for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ top_P_abs_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['ext_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['ext_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Extractive_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

------ top_P_abs_summary-------
Average Rouge1 Score:  20.78922596767385
Average Rouge2 Score:  1.687723939962558
------ Extractive_summary-------
Average Rouge1 Score:  13.610906174527997
Average Rouge2 Score:  1.2735726147453659
------ Beams_summary-------
Average Rouge1 Score:  13.247162362541905
Average Rouge2 Score:  1.198764949141089
------ Beams_summary-------
Average Rouge1 Score:  12.994571080754296
Average Rouge2 Score:  1.2201822951746712


In [24]:
test.to_csv('test_ext.csv', sep='\t')